# Data Retrieval

In [1]:
import pandas as pd
import csv
import re
import statsmodels.api as sm
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.ensemble import RandomForestRegressor

In [2]:
# File path for cities: San Francisco(sf), 
# Okaland(ok), Santa Clara(sc), Santa Cruze(scz)

file_url = '/Users/xzhou/github/project_archives/files_airbnb/{}/calendar.csv'

sf_file = file_url.format('sf_airbnb')
ok_file = file_url.format('ok_airbnb')
sc_file = file_url.format('sc_airbnb')
scz_file = file_url.format('scz_airbnb')

In [3]:
# Loaded price by date information for each city

pricing_sf = pd.read_csv(sf_file, parse_dates=['date'])
pricing_ok = pd.read_csv(ok_file, parse_dates=['date'])
pricing_sc = pd.read_csv(sc_file, parse_dates=['date'])
pricing_scz = pd.read_csv(scz_file, parse_dates=['date'])

print(pricing_sf.shape, pricing_ok.shape, pricing_sc.shape, pricing_scz.shape)

(2420680, 4) (1057770, 4) (2068820, 4) (573050, 4)


In [4]:
# Concatenate four files into one
# Created keys for each file for future references

frames = [pricing_sf, pricing_ok, pricing_sc, pricing_scz]
keys = ['pricing_sf', 'pricing_ok', 'pricing_sc', 'pricing_scz']

df_pricing = pd.concat(frames, keys=keys)

df_pricing.head(10)

listing_id       date available price
pricing_sf 0         958 2019-08-05         f   NaN
           1         958 2019-08-04         f   NaN
           2         958 2019-08-03         f   NaN
           3         958 2019-08-02         f   NaN
           4         958 2019-08-01         f   NaN
           5         958 2019-07-31         f   NaN
           6         958 2019-07-30         f   NaN
           7         958 2019-07-29         f   NaN
           8         958 2019-07-28         f   NaN
           9         958 2019-07-27         f   NaN

# Data Clean up

## Filter down to data with pricing information

In [7]:
# Filter to houses that are available and have listing prices

df_pricing_smaller = df_pricing[df_pricing['available']=='t']
df_pricing_smaller = df_pricing_smaller.copy()

print('Dimension for df_pricing_smaller: ', df_pricing_smaller.shape)
print ('\nNumber of Unique ids: ', df_pricing_smaller.listing_id.nunique())

Dimension for df_pricing_smaller:  (2853357, 4)

Number of Unique ids:  14770


In [8]:
df_pricing_smaller.columns

Index(['listing_id', 'date', 'available', 'price'], dtype='object')

In [9]:
display( df_pricing_smaller.date.describe(include='all'))

count                 2853357
unique                    389
top       2018-11-03 00:00:00
freq                    10393
first     2018-08-06 00:00:00
last      2019-08-29 00:00:00
Name: date, dtype: object

## Format Conversion and Outlier Removal

In [11]:
# Noted currencies are formated as strings and need to convert to float

def currency2float(string):
    """
    Convert currency to float
    """
    float_ = float(str(string).strip('$').replace(',',''))
    return float_

In [12]:
# Converted price to float format

df_pricing_smaller['price'] = df_pricing_smaller['price'].apply(
                              lambda x: currency2float(x))

In [17]:
# Removed outliers based on 'price'. Use 5% and 95% percentiles as cutoff

five_percentile = np.percentile(df_pricing_smaller.price, 5)
ninety_five_percentile = np.percentile(df_pricing_smaller.price, 95)

df_pricing_smaller = df_pricing_smaller[(
                         df_pricing_smaller.price>=five_percentile)
                        & (df_pricing_smaller.price<=ninety_five_percentile)]

print ('df_pricing_smaller dimensions: ', df_pricing_smaller.shape)

df_pricing_smaller dimensions:  (1931274, 4)


## Filter Data

In [18]:
# Added 'month' column to extract of month of date
# Added 'day_of_week' column to extact day of week. 
# Default format: Monday=0, ..., Sunday=6

df_pricing_smaller['month'] = df_pricing_smaller['date'].dt.month
df_pricing_smaller['day_of_week'] = df_pricing_smaller['date'].dt.weekday


print('df_pricing_smaller dimensions', df_pricing_smaller.shape)
display(df_pricing_smaller.head())

df_pricing_smaller dimensions (1931274, 6)


listing_id       date available  price  month  day_of_week
pricing_sf 63         958 2019-05-02         t  181.0      5            3
           64         958 2019-05-01         t  181.0      5            2
           65         958 2019-04-30         t  181.0      4            1
           66         958 2019-04-29         t  181.0      4            0
           67         958 2019-04-28         t  181.0      4            6

In [19]:
display(df_pricing_smaller.describe(include ='all'))

listing_id                 date available         price  \
count   1.931274e+06              1931274   1931274  1.931274e+06   
unique           NaN                  389         1           NaN   
top              NaN  2018-11-03 00:00:00         t           NaN   
freq             NaN                 7096   1931274           NaN   
first            NaN  2018-08-06 00:00:00       NaN           NaN   
last             NaN  2019-08-29 00:00:00       NaN           NaN   
mean    1.389968e+07                  NaN       NaN  1.433284e+02   
std     8.724262e+06                  NaN       NaN  5.813113e+01   
min     9.580000e+02                  NaN       NaN  6.500000e+01   
25%     5.914638e+06                  NaN       NaN  9.400000e+01   
50%     1.460646e+07                  NaN       NaN  1.310000e+02   
75%     2.173651e+07                  NaN       NaN  1.860000e+02   
max     2.815247e+07                  NaN       NaN  2.890000e+02   

               month   day_of_week  
count   1.931274e+06  1.931274e+06  
unique           NaN           NaN  
top              NaN           NaN  
freq             NaN           NaN  
first            NaN           NaN  
last             NaN           NaN  
mean    6.550739e+00  2.986810e+00  
std     3.647660e+00  2.003891e+00  
min     1.000000e+00  0.000000e+00  
25%     3.000000e+00  1.000000e+00  
50%     7.000000e+00  3.000000e+00  
75%     1.000000e+01  5.000000e+00  
max     1.200000e+01  6.000000e+00

In [69]:
df_by_month_day = df_pricing_smaller.groupby(['listing_id', 'month', 'day_of_week'])  \
                                    .price.mean()  \
                                    .reset_index()
df_by_month_day.head()

listing_id  month  day_of_week  price
0         958      1            0  180.0
1         958      1            1  179.5
2         958      1            2  179.5
3         958      1            3  179.0
4         958      1            4  183.0

In [80]:
df_month_day_counts = df_by_month_day.groupby(['listing_id'])  \
                                     .price.count()  \
                                 .reset_index()
df_month_day_counts.rename(columns = {'price':'counts'}, inplace=True)
df_month_day_counts.head()

listing_id  counts
0         958      48
1        3083      28
2        5739      12
3        5858      84
4        7918      63

In [81]:
# Find out listings that have full price information (12 months and 7 days/week)

listing_with_12m_7d_price = list(df_month_day_counts[
                                df_month_day_counts.counts==12*7]
                                .listing_id)

In [83]:
df_pricing_available = df_pricing_smaller[df_pricing_smaller.listing_id.isin(
                                          listing_with_12m_7d_price)]
df_pricing_available.shape

(655132, 6)

In [85]:
df_pricing_available.describe(include='all')

listing_id                 date available          price  \
count   6.551320e+05               655132    655132  655132.000000   
unique           NaN                  389         1            NaN   
top              NaN  2019-05-06 00:00:00         t            NaN   
freq             NaN                 1883    655132            NaN   
first            NaN  2018-08-06 00:00:00       NaN            NaN   
last             NaN  2019-08-29 00:00:00       NaN            NaN   
mean    1.362489e+07                  NaN       NaN     139.006151   
std     8.822227e+06                  NaN       NaN      55.700680   
min     5.858000e+03                  NaN       NaN      65.000000   
25%     5.721902e+06                  NaN       NaN      90.000000   
50%     1.341834e+07                  NaN       NaN     128.000000   
75%     2.173730e+07                  NaN       NaN     179.000000   
max     2.815247e+07                  NaN       NaN     289.000000   

                month    day_of_week  
count   655132.000000  655132.000000  
unique            NaN            NaN  
top               NaN            NaN  
freq              NaN            NaN  
first             NaN            NaN  
last              NaN            NaN  
mean         6.426778       2.993769  
std          3.488229       2.003698  
min          1.000000       0.000000  
25%          3.000000       1.000000  
50%          6.000000       3.000000  
75%         10.000000       5.000000  
max         12.000000       6.000000

In [87]:
# Group by month by weekday, and calculate an average pricing

df_pricing_by_month_weekday = df_pricing_available.groupby(  \
                              ['listing_id', 'month', 'day_of_week']) \
                              .price.mean()  \
                              .reset_index()
df_pricing_by_month_weekday.head()

listing_id  month  day_of_week  price
0        5858      1            0  235.0
1        5858      1            1  235.0
2        5858      1            2  235.0
3        5858      1            3  235.0
4        5858      1            4  235.0

In [88]:
df_pricing_by_month_weekday.describe(include='all')

listing_id          month    day_of_week          price
count  1.581720e+05  158172.000000  158172.000000  158172.000000
mean   1.360873e+07       6.500000       3.000000     138.868869
std    8.823256e+06       3.452063       2.000006      55.536787
min    5.858000e+03       1.000000       0.000000      65.000000
25%    5.641077e+06       3.750000       1.000000      90.000000
50%    1.341834e+07       6.500000       3.000000     127.000000
75%    2.173651e+07       9.250000       5.000000     179.000000
max    2.815247e+07      12.000000       6.000000     289.000000

In [89]:
# Check for null values if any

df_pricing_by_month_weekday.isnull().sum()

listing_id     0
month          0
day_of_week    0
price          0
dtype: int64

In [90]:
# Rename the average price column to avoid confusion

df_pricing_by_month_weekday.rename(columns={'price':'price_by_month_week'}, inplace=True)

In [91]:
df_pricing_by_month_weekday.shape

(158172, 4)

In [92]:
df_pricing_by_month_weekday.head()

listing_id  month  day_of_week  price_by_month_week
0        5858      1            0                235.0
1        5858      1            1                235.0
2        5858      1            2                235.0
3        5858      1            3                235.0
4        5858      1            4                235.0

In [93]:
def get_month(number):
    """
    Given a number, return the name of month.
    
    """
    month_lookup = {1:'Jan.', 2:'Feb.', 3:'Mar.', 4:'Apr.', 5:'May.', 6:'Jun.',
                    7:'Jul.', 8:'Aug.', 9:'Sep.', 10:'Oct.', 11:'Nov.', 12:'Dec.'}
    
    return month_lookup[number]

In [94]:
def get_weekdays(number):
    """
    Given a number, return the name of month.
    
    """
    weekday_lookup = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 
                      4:'Friday', 5:'Saturday', 6:'Sunday'}
    
    return weekday_lookup[number]

In [95]:
df_pricing_by_month_weekday['month'] = df_pricing_by_month_weekday['month']  \
                                       .apply(lambda x: get_month(x))

In [96]:
df_pricing_by_month_weekday['day_of_week'] = df_pricing_by_month_weekday['day_of_week']  \
                                             .apply(lambda x: get_weekdays(x))

In [97]:
df_pricing_final = pd.get_dummies(df_pricing_by_month_weekday, 
                                  prefix='',
                                  prefix_sep='',
                                  columns=['month', 'day_of_week'])

In [98]:
df_pricing_final.columns

Index(['listing_id', 'price_by_month_week', 'Apr.', 'Aug.', 'Dec.', 'Feb.',
       'Jan.', 'Jul.', 'Jun.', 'Mar.', 'May.', 'Nov.', 'Oct.', 'Sep.',
       'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday'],
      dtype='object')

In [99]:
df_pricing_final.shape

(158172, 21)

In [100]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'

pkl_path = pkl_file.format('listings_with_selected_features')

df_listings_reviews = pd.read_pickle(pkl_path)
df_listings_reviews.head()

id  bedrooms  cleaning_fee  room_type_Entire home/apt  reviews_per_month  \
0   958       1.0       2.00000                          1               1.38   
1  5858       2.0       2.00000                          1               0.99   
2  7918       1.0       1.69897                          0               0.16   
3  8142       1.0       1.69897                          0               0.15   
4  8567       2.0       2.09691                          1               0.27   

   city_San Francisco  city_Oakland  bathrooms  accommodates  \
0                   1             0        1.0             3   
1                   1             0        1.0             5   
2                   1             0        4.0             2   
3                   1             0        4.0             2   
4                   1             0        1.0             6   

   number_of_reviews        ...         neighbourhood_cleansed_Palo Alto  \
0                152        ...                                        0   
1                112        ...                                        0   
2                 17        ...                                        0   
3                  7        ...                                        0   
4                 30        ...                                        0   

   host_is_superhost_f  amenities_Lock on bedroom door  \
0                    0                               0   
1                    1                               0   
2                    1                               1   
3                    1                               1   
4                    1                               0   

   amenities_Private entrance  amenities_Familykid friendly  \
0                           1                             1   
1                           1                             1   
2                           1                             0   
3                           1                             1   
4                           0                             1   

   district_Santa Clara  neighbourhood_cleansed_San Jose  \
0                     0                                0   
1                     0                                0   
2                     0                                0   
3                     0                                0   
4                     0                                0   

   host_identity_verified_f   ref_district  ref_jurisdiction  
0                         0  San Francisco     SAN FRANCISCO  
1                         0  San Francisco     SAN FRANCISCO  
2                         0  San Francisco     SAN FRANCISCO  
3                         0  San Francisco     SAN FRANCISCO  
4                         1  San Francisco     SAN FRANCISCO  

[5 rows x 33 columns]

In [101]:
df_listings_reviews.shape

(10352, 33)

In [102]:
df_final = pd.merge(left=df_pricing_final, 
                    right=df_listings_reviews,
                    how='inner',
                    left_on='listing_id',
                    right_on='id')

In [103]:
df_final.shape

(95424, 54)

In [104]:
df_final.isnull().sum()

listing_id                                  0
price_by_month_week                         0
Apr.                                        0
Aug.                                        0
Dec.                                        0
Feb.                                        0
Jan.                                        0
Jul.                                        0
Jun.                                        0
Mar.                                        0
May.                                        0
Nov.                                        0
Oct.                                        0
Sep.                                        0
Friday                                      0
Monday                                      0
Saturday                                    0
Sunday                                      0
Thursday                                    0
Tuesday                                     0
Wednesday                                   0
id                                

In [105]:
df_final.shape

(95424, 54)

In [106]:
df_final.columns

Index(['listing_id', 'price_by_month_week', 'Apr.', 'Aug.', 'Dec.', 'Feb.',
       'Jan.', 'Jul.', 'Jun.', 'Mar.', 'May.', 'Nov.', 'Oct.', 'Sep.',
       'Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
       'Wednesday', 'id', 'bedrooms', 'cleaning_fee',
       'room_type_Entire home/apt', 'reviews_per_month', 'city_San Francisco',
       'city_Oakland', 'bathrooms', 'accommodates', 'number_of_reviews',
       'host_listings_count', 'review_scores_rating_log10', 'host_yrs',
       'guests_included', 'calculated_host_listings_count',
       'review_scores_location_log10', 'beds', 'yrs_since_first_review',
       'host_response_rate', 'amenities_Air conditioning',
       'review_scores_value_log10', 'jurisdiction_names_Santa Cruz County, CA',
       'amenities_Cable TV', 'neighbourhood_cleansed_Palo Alto',
       'host_is_superhost_f', 'amenities_Lock on bedroom door',
       'amenities_Private entrance', 'amenities_Familykid friendly',
       'district_Santa Clara', 'n

In [107]:
# Drop duplicate columns

df_final.drop(['id'], axis=1, inplace=True)

In [108]:
df_final.shape

(95424, 53)

In [109]:
# Create correlation matrix
corr_matrix = df_final.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.90
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]

to_drop

['review_scores_value_log10']

In [110]:
# Drop features that have high correlation

df_final.drop(to_drop, axis=1, inplace=True)

In [111]:
df_final.shape

(95424, 52)

In [112]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'
pkl_path = pkl_file.format('listings_seasonality')

with open(pkl_path, 'wb') as file:
        pd.to_pickle(df_final, file)

In [ ]:
pkl_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.pkl'
pkl_path = pkl_file.format('listings_seasonality')

try: 
    with open(pkl_path, 'rb') as file:
        df_final = pd.read_pickle(file)
except:
    with open(pkl_path, 'wb') as file:
        pd.to_pickle(df_final, file)

In [113]:
# Perform multiple modeling for price prediction

X = df_final.drop(['price_by_month_week', 'ref_district', 'ref_jurisdiction'], axis=1)
y = df_final.loc[:, ['listing_id', 'price_by_month_week']]

In [114]:
grp_by_listing_ids = df_final.groupby('listing_id').first().reset_index().listing_id

In [118]:
listing_ids = list (grp_by_listing_ids)

In [119]:
num_of_grps = len(grp_by_listing_ids)

training_ids = np.random.choice(listing_ids, int(num_of_grps*0.75), replace=False)
print(len(training_ids))
training_ids

852


array([25628339,  5193718,   717000,  5861266,    69410, 20695034,
       10912061, 10233191,  8192494,  2182169,  7636885, 20716986,
        4251101,  4032327,  5005798,   427707,  1241029, 15194320,
         307827,  2008590,  2829131, 18022892, 12872051, 11983474,
        7459204,  7887588, 15939487,  1914602,  7858199,  6214853,
        3541007,   745925, 24470284,  4297189,  5058777,  3890704,
       21345876,  1442027,  1722167,  5116060,  5008287,  3022391,
       24783410, 24100944,  4120530, 13359333, 10595590,  3238531,
        2465868,   443908, 14534564, 19725513, 17026443,  4119973,
       21295356, 22112477,  7887030,  1349884,  1162640, 17076076,
       25580922, 11591266, 26171011, 26631967,   226594, 10413713,
       12212126,    39174, 13014186, 15624712, 14885810, 10316398,
        1459083, 10886381, 23672681, 26316944,  6186641, 14311906,
       20148137, 13465756,  4580381, 21647837, 11372655,  3706161,
        1873261, 17301015, 13360872,  8764158,  7844355, 22534

In [120]:
testing_ids = set(grp_by_listing_ids) - set (training_ids)
len(testing_ids)

284

In [121]:
X_train = X[X['listing_id'].isin(training_ids)].drop('listing_id', axis=1)
X_test = X[X['listing_id'].isin(testing_ids)].drop('listing_id', axis=1)
y_train = y[y['listing_id'].isin(training_ids)].drop('listing_id', axis=1).values.ravel()
y_test = y[y['listing_id'].isin(testing_ids)].drop('listing_id', axis=1).values.ravel()

print(len(X_train), len(X_test), len(y_train), len(y_test))

71568 23856 71568 23856


In [122]:
ss = StandardScaler()
X_train_trans = ss.fit_transform(X_train)
X_test_trans = ss.transform (X_test)

In [123]:
# Model-LR

model=sm.OLS(y_train, sm.add_constant(X_train_trans))
fit=model.fit()
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.475
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     1404.
Date:                Fri, 15 Feb 2019   Prob (F-statistic):               0.00
Time:                        23:34:47   Log-Likelihood:            -3.6753e+05
No. Observations:               71568   AIC:                         7.351e+05
Df Residuals:                   71521   BIC:                         7.356e+05
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        135.7346      0.156    868.907      0.000     135.428     136.041
x1         -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x2         -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x3         -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x4         -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x5         -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x6         -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x7         -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x8         -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x9         -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x10        -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x11        -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x12        -4.823e+11   3.07e+12     -0.157      0.875   -6.49e+12    5.53e+12
x13         1.472e+12   6.56e+12      0.224      0.823   -1.14e+13    1.43e+13
x14         1.472e+12   6.56e+12      0.224      0.823   -1.14e+13    1.43e+13
x15         1.472e+12   6.56e+12      0.224      0.823   -1.14e+13    1.43e+13
x16         1.472e+12   6.56e+12      0.224      0.823   -1.14e+13    1.43e+13
x17         1.472e+12   6.56e+12      0.224      0.823   -1.14e+13    1.43e+13
x18         1.472e+12   6.56e+12      0.224      0.823   -1.14e+13    1.43e+13
x19         1.472e+12   6.56e+12      0.224      0.823   -1.14e+13    1.43e+13
x20           11.2099      0.218     51.508      0.000      10.783      11.636
x21            5.2482      0.188     27.944      0.000       4.880       5.616
x22           16.4571      0.215     76.410      0.000      16.035      16.879
x23           -1.4396      0.281     -5.124      0.000      -1.990      -0.889
x24           10.0298      0.709     14.138      0.000       8.639      11.420
x25           -1.3050      0.545     -2.395      0.017      -2.373      -0.237
x26           -2.5827      0.191    -13.523      0.000      -2.957      -2.208
x27           14.9466      0.320     46.666      0.000      14.319      15.574
x28           -0.8375      0.298     -2.815      0.005      -1.421      -0.254
x29            4.1191      0.222     18.542      0.000       3.684       4.554
x30           -1.8471      0.382     -4.833      0.000      -2.596      -1.098
x31           -0.5683      0.206     -2.763      0.006      -0.971      -0.165
x32            5.1689      0.186     27.802      0.000       4.805       5.533
x33           -1.3291      0.226     -5.869      0.000      -1.773      -0.885
x34            5.2560      0.381     13.792      0.000       4.509       6.003
x35           -4.8797      0.305    -15.976      0.000      -5.478      -4.281
x3

In [124]:
# Model-Random Forest

rf = RandomForestRegressor(n_estimators=500, 
                           criterion='mse', 
                           random_state=42, 
                           n_jobs=-1)
rf.fit(X_train_trans, y_train.ravel())

y_train_pred = rf.predict(X_train_trans)
y_test_pred = rf.predict(X_test_trans)

print('MSE train: %.3f, test: %.3f' % (
        mean_squared_error(y_train, y_train_pred),
        mean_squared_error(y_test, y_test_pred)))
print('R^2 train: %.3f, test: %.3f' % (
        r2_score(y_train, y_train_pred),
        r2_score(y_test, y_test_pred)))

MSE train: 1.428, test: 1694.647
R^2 train: 1.000, test: 0.414


In [ ]:
RMSE = 1043.202**0.5
RMSE

In [ ]:
df_final.listing_id.nunique()

In [ ]:
# Double check on price range. The range is as expected

y.describe(include='all')

In [ ]:
# Map feature importance with labels

feature_labels = X.columns
feature_tuples = zip (feature_labels, rf.feature_importances_)
feature_lists = [list(feature_tuple) for feature_tuple in feature_tuples]

In [ ]:
# Order features based on importance

feature_lists.sort(key=lambda x: abs(x[1]), reverse=True)
feature_lists

In [ ]:
features = [list_[0] for list_ in feature_lists]
features

In [ ]:
columns =['listing_id']
columns.extend(features)
columns.extend(['ref_district', 'ref_jurisdiction', 'price_by_month_week'])

In [ ]:
df_data = df_final[columns]
df_data.head()

In [ ]:
df_data.shape

In [ ]:
df_data.rename(columns={'price_by_month_week':'price'}, inplace=True)

In [ ]:
df_data.isnull().sum()

In [ ]:
temp_df = df_data[['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']].copy()

days = temp_df.apply(lambda x: x.idxmax(), axis=1)
days

In [ ]:
csv_file = '/Users/xzhou/github/project_archives/files_airbnb/{}.csv'

csv_path = csv_file.format('airbnb_data')

df_data.to_csv(csv_path)

In [ ]:
# This is to create visual to show seasonality

df_visual = pd.merge(left=df_pricing_by_month_weekday, 
                    right=df_listings_reviews,
                    how='inner',
                    left_on='listing_id',
                    right_on='id')

In [ ]:
df_visual.shape

In [ ]:
df_visual.columns

In [ ]:
df_visual_entire_apartment = df_visual[df_visual['room_type_Entire home/apt']==1 ]
df_visual_single_room = df_visual[df_visual['room_type_Entire home/apt']!=1 ]

print(df_visual_entire_apartment.shape, df_visual_single_room.shape )

In [ ]:
df_visual_entire_apartment.ref_district.describe

In [ ]:
df_visual_entire_apartment_sf = df_visual_entire_apartment[df_visual_entire_apartment['ref_district']=='San Francisco']
df_visual_entire_apartment_scz = df_visual_entire_apartment[df_visual_entire_apartment['ref_district']=='Santa Clara']

print(df_visual_entire_apartment_sf.shape, df_visual_entire_apartment_scz.shape )

In [ ]:
df_visual_entire_apartment_sf.id.nunique()

In [ ]:
df_visual_entire_apartment_scz.id.nunique()

In [ ]:
# Weekly pattern for entire apartment

df_week_entire_apartment = df_visual_entire_apartment.groupby(['ref_district', 'day_of_week'])  \
                                                     .price_by_month_week.mean()
df_week_entire_apartment

In [ ]:
# Weekly pattern for single room

df_week_single_room = df_visual_single_room.groupby(['ref_district', 'day_of_week'])  \
                                           .price_by_month_week.mean()
df_week_single_room

In [ ]:
# Montly Pattern for entire apartment

df_visual_entire_apartment.groupby(['ref_district', 'month']).price_by_month_week.mean()

In [ ]:
# Montly Pattern for single room

df_visual_single_room.groupby(['ref_district', 'month']).price_by_month_week.mean()